<a href="https://colab.research.google.com/github/JamCourage/Tone-of-Sustainability-Report/blob/main/2.%20FinBERT_calculate%20tone/crawler_finbert_for_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 設定路徑
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab Notebooks/ESG_FinBERT')  # 切換目錄(去檔案找->先掛接雲端->再複製資料夾路徑)
os.listdir()  # 看目錄內有什麼


Mounted at /content/drive


['ESG_report_111',
 'finbert_example.ipynb',
 'ESG_report_110',
 'ESG_report_107',
 'file_crawer.ipynb',
 'pdf_crawler.ipynb',
 'pdf_crawler_for_one.ipynb']

**1：整理公司股票代碼**

In [ ]:
# 1-1：擷取資料夾內所有PDF檔案名稱
pdf_file_names = os.listdir("ESG_report_107")  # 儲存資料夾內所有檔案名稱(*.pdf)為list
print(pdf_file_names[0])
print(len(pdf_file_names))  # 150份

1102_107.pdf
150


In [ ]:
# 1-2：要裝擷取出的文字的dataframe
import pandas as pd

# 創建一個空的 DataFrame
ESG_reports_df = pd.DataFrame(columns=['sentence', 'E_neutral', 'E_positive', 'E_negative', 'S_neutral', 'S_positive', 'S_negative', 'G_neutral', 'G_positive', 'G_negative', 'non_neutral', 'non_positive', 'non_negative'])

# 打印創建的 DataFrame
print(ESG_reports_df)


Empty DataFrame
Columns: [sentence, E_neutral, E_positive, E_negative, S_neutral, S_positive, S_negative, G_neutral, G_positive, G_negative, non_neutral, non_positive, non_negative]
Index: []


2: **安裝FinBERT情緒分類模型 & FinBERT-ESG主題分類模型**

In [ ]:
# 2-1:
# 安裝transformers
%pip install transformers

# import套件--part 1情緒分類
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

# Download and load the fine-tuned FinBERT sentiment classification model and the vocabulary (FinVocab, the finance vocabulary discussed earlier) from Hugging Face.
finbert_sent = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)  # fine-tuned FinBERT sentiment classification model
tokenizer_sent = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')  # the vocabulary (FinVocab, the finance vocabulary)

#################################################################
# part2：利用finbert_esg9classification將ESG句子分類為9大面向
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert_esg = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories',num_labels=9)  # 選用finbert之9大ESG分類模型
tokenizer_esg = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')  # pre-train
nlp_esg = pipeline("text-classification", model=finbert_esg, tokenizer=tokenizer_esg)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**3: 擷取PDF文字、文字前處理、斷句、FinBERT兩大模型、紀錄結果**

In [ ]:
# 3-1：指定讀取某一pdf檔案(不同)
!pip install pdfplumber
import pdfplumber  # 讀取pdf套件

# 打開PDF檔案
for i, d in enumerate(pdf_file_names[7:8]):  # 一個一個pdf打開 d="1102_107.pdf"
  text = ''  # 裝該pdf的純文字內容

  # 先從檔案名稱中擷取company_id
  #part1, part2 = d.split('_')
  #company_id = part1

  # 開始提取PDF內純文字
  path = "ESG_report_107/" + d  # 各pdf路徑
  with pdfplumber.open(path) as pdf:
    for page in pdf.pages:  # 一頁一頁看
      text += page.extract_text()  # 提取純文字內容

  # print(company_id, text[0:2000], len(text))  # check各pdf擷取狀況
  # print(company_id, len(text))


  ################################## 3-2：刪除特殊字元，並以空白取代(保留英文字母大小寫 .,!?)############################################
  import re

  # 自定義刪除特殊字元函數，輸入值為「未清理之文字」，輸出為「已清理之文字」
  def remove_special_characters(text):
    # 定義正規表達式(regular expression)，匹配「字母以外的任何字符(特殊字元)，但保留,.!?」
    pattern = re.compile('[^a-zA-Z,.\!?]+')  # 匹配非(英文字母大小寫、正常標點符號)

    # 將正規表達式應用於文本
    cleaned_text = pattern.sub(' ', text)  # 匹配的特殊字元以空格取代

    return cleaned_text


  # 使用 apply 函數將 remove_special_characters 函數應用於 'text' 中的每個元素
  cleaned_text = remove_special_characters(text)

  if len(cleaned_text) == 0:
    continue

  ###########################################################################################################################################
  # 3-3: 斷句、分類、紀錄
  from nltk import word_tokenize, sent_tokenize

  for num, sent in enumerate(sent_tokenize(cleaned_text)):  # 斷句
    if len(sent) > 400:
      sent_r = sent[:400]  # 擷取前400字
    else:
      sent_r = sent

    sents = []
    sents.append(sent_r)  # 將句子以list形式傳入--只有一句

    ############################################### 3-4 part 1情緒分類--一句一句標情緒#######################
    # 最大不能超過512張量，否則切斷
    inputs = tokenizer_sent(sents, return_tensors="pt", padding=True, truncation=True, max_length=512)  # tokenize the sentences (pt:PyTorch輸出轉換形式):將文本 sent 轉換為 BERT 模型的輸入格式，並返回 PyTorch 張量。
    outputs = finbert_sent(**inputs)[0]  # 輸出 process them in FinBERT

    # Assign each sentence to its most probable sentiment category.
    labels = {0:'neutral', 1:'positive', 2:'negative'}

    for idx, s in enumerate(sents):  # idx=index s=sentence
      sentiment = labels[np.argmax(outputs.detach().numpy()[idx])]  # part1: neutral/positive/negative

      ################################ 3-4 part 2主題分類########################################
      results_esg = nlp_esg(sents)
      esg_class_detail = results_esg[0]['label']  # part2: 九大分類

      if esg_class_detail == 'Climate Change' or  esg_class_detail == 'Natural Capital' or esg_class_detail == 'Pollution & Waste':
        esg_class = 'E'
      elif esg_class_detail == 'Human Capital' or  esg_class_detail == 'Product Liability' or esg_class_detail == 'Community Relations':
        esg_class = 'S'
      elif esg_class_detail == 'Corporate Governance' or  esg_class_detail == 'Business Ethics & Values':
        esg_class = 'G'
      else:
        esg_class = 'non_ESG'

      #############################3-5 整合part1 and part2##################################
      # 共有12種情況
      # 'E_neutral', 'E_positive', 'E_negative', 'S_neutral', 'S_positive', 'S_negative', 'G_neutral', 'G_positive', 'G_negative', 'non_neutral', 'non_positive', 'non_negative'

      data_to_add = {'sentence': sents[0]}  # 把句子填入dataframe中(不同處)
      ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)

      if esg_class == 'E' and sentiment == 'neutral':
        ESG_reports_df.loc[num, 'E_neutral'] = 1
      elif esg_class == 'E' and sentiment == 'positive':
        ESG_reports_df.loc[num, 'E_positive'] = 1
      elif esg_class == 'E' and sentiment == 'negative':
        ESG_reports_df.loc[num, 'E_negative'] = 1
      elif esg_class == 'S' and sentiment == 'neutral':
        ESG_reports_df.loc[num, 'S_neutral'] = 1
      elif esg_class == 'S' and sentiment == 'positive':
        ESG_reports_df.loc[num, 'S_positive'] = 1
      elif esg_class == 'S' and sentiment == 'negative':
        ESG_reports_df.loc[num, 'S_negative'] = 1
      elif esg_class == 'G' and sentiment == 'neutral':
        ESG_reports_df.loc[num, 'G_neutral'] = 1
      elif esg_class == 'G' and sentiment == 'positive':
        ESG_reports_df.loc[num, 'G_positive'] = 1
      elif esg_class == 'G' and sentiment == 'negative':
        ESG_reports_df.loc[num, 'G_negative'] = 1
      elif esg_class == 'non_ESG' and sentiment == 'neutral':
        ESG_reports_df.loc[num, 'non_neutral'] = 1
      elif esg_class == 'non_ESG' and sentiment == 'positive':
        ESG_reports_df.loc[num, 'non_positive'] = 1
      elif esg_class == 'non_ESG' and sentiment == 'negative':
        ESG_reports_df.loc[num, 'non_negative'] = 1
      # print(s, '------', labels[np.argmax(outputs.detach().numpy()[idx])])

# 輸出
ESG_reports_df.to_excel('ESG_sent_and_class_1102_107.xlsx', index=True)

串流輸出內容已截斷至最後 5000 行。
<ipython-input-8-709bc79448dd>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-8-709bc79448dd>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-8-709bc79448dd>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-8-709bc79448dd>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-

In [ ]:
print(ESG_reports_df.loc[0, 'company_id'])
print(ESG_reports_df)

1102
  company_id E_neutral E_positive E_negative S_neutral S_positive S_negative  \
0       1102       905        253         32       629        239         16   
1       1216        37          5          1        61          8          0   
2       1301       451        105         23       673        180         35   
3       1303       359         96         10       615        191         27   

  G_neutral G_positive G_negative non_neutral non_positive non_negative  
0       191         29          2         431           21            3  
1         7          1          0          40            0            0  
2       272         38         12        1184           42           16  
3       262         36          8         824           38           10  


In [ ]:
# 1-3：讀取每一份PDF檔案
!pip install pdfplumber
import pdfplumber  # 讀取pdf套件

# 打開PDF檔案
for d in pdf_file_names:  # 一個一個pdf打開 d="1102_107.pdf"
  text = ''  # 裝該pdf的純文字內容

  # 先從檔案名稱中擷取company_id
  part1, part2 = d.split('_')
  company_id = part1

  # 開始提取PDF內純文字
  path = "ESG_report_107/" + d  # 各pdf路徑
  with pdfplumber.open(path) as pdf:
    for page in pdf.pages:  # 一頁一頁看
      text += page.extract_text()  # 提取純文字內容

  # print(company_id, text[0:2000], len(text))  # check各pdf擷取狀況
  # print(company_id, len(text))

  # 將該份PDF的公司id、文字存入dataframe中
  data_to_add = {'company_id': company_id, 'texts': text}
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
print(ESG_reports_df.loc[0, 'text'])

<ipython-input-5-e75229b52f2c>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-5-e75229b52f2c>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-5-e75229b52f2c>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-5-e75229b52f2c>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-5-e75229b52f2c>:24: F

KeyboardInterrupt: 

In [ ]:
print(ESG_reports_df.loc['1476', 'text'])
print(len(ESG_reports_df['text']))

2: 文字前處理

In [ ]:
# 2-1：刪除特殊字元，並以空白取代(保留英文字母大小寫 .,!?)
import re

# 自定義刪除特殊字元函數，輸入值為「未清理之文字」，輸出為「已清理之文字」
def remove_special_characters(text):
  # 定義正規表達式(regular expression)，匹配「字母以外的任何字符(特殊字元)，但保留,.!?」
  pattern = re.compile('[^a-zA-Z,.\!?]+')  # 匹配非(英文字母大小寫、正常標點符號)

  # 將正規表達式應用於文本
  cleaned_text = pattern.sub(' ', text)  # 匹配的特殊字元以空格取代

  return cleaned_text


# 使用 apply 函數將 remove_special_characters 函數應用於 'text' 中的每個元素
ESG_reports_df['cleaned_text'] = ESG_reports_df['text'].apply(remove_special_characters)

# 印出結果
#print(tweet_stock_df['text'])
print(ESG_reports_df.loc['1476', 'cleaned_text'])

In [ ]:
# 2-2: 使用 strip 方法去除每一列的空格，然後使用 dropna 刪除包含空值的整行
ESG_reports_df = ESG_reports_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # 對每一個元素應用 strip 方法
ESG_reports_df = ESG_reports_df.dropna()

In [ ]:
# 2-3: 使用 apply 方法和 lambda 函數檢查 'Text' 欄位中值的長度，然後使用 dropna 刪除包含空值的整行
ESG_reports_df['cleaned_text'] = ESG_reports_df['cleaned_text'].apply(lambda x: x if len(str(x).strip()) > 0 else np.nan)
ESG_reports_df = ESG_reports_df.dropna()

In [ ]:
print(len(ESG_reports_df['text']))

3: FinBERT

In [ ]:
# 3-1:
# 安裝transformers
%pip install transformers

# import套件--part 1情緒分類
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

# Download and load the fine-tuned FinBERT sentiment classification model and the vocabulary (FinVocab, the finance vocabulary discussed earlier) from Hugging Face.
finbert_sent = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)  # fine-tuned FinBERT sentiment classification model
tokenizer_sent = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')  # the vocabulary (FinVocab, the finance vocabulary)

#################################################################
# part2：利用finbert_esg9classification將ESG句子分類為9大面向
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert_esg = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories',num_labels=9)  # 選用finbert之9大ESG分類模型
tokenizer_esg = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')  # pre-train
nlp_esg = pipeline("text-classification", model=finbert_esg, tokenizer=tokenizer_esg)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# 3-2: 斷句、分類、紀錄
from nltk import word_tokenize, sent_tokenize

for i, txt in enumerate(ESG_reports_df['cleaned_text']):
  for sent in sent_tokenize(txt):  # 斷句
    if len(sent) > 200:
      sent_r = sent[:200]  # 擷取前200字
    else:
      sent_r = sent

    sents = []
    sents.append(sent_r)  # 將句子以list形式傳入--只有一句

    ############################################### part 1情緒分類--一句一句標情緒#######################
    # 最大不能超過512張量，否則切斷
    inputs = tokenizer_sent(sents, return_tensors="pt", padding=True, truncation=True, max_length=512)  # tokenize the sentences (pt:PyTorch輸出轉換形式):將文本 sent 轉換為 BERT 模型的輸入格式，並返回 PyTorch 張量。
    outputs = finbert_sent(**inputs)[0]  # 輸出 process them in FinBERT

    # Assign each sentence to its most probable sentiment category.
    labels = {0:'neutral', 1:'positive', 2:'negative'}

    for idx, s in enumerate(sents):  # idx=index s=sentence
      sentiment = labels[np.argmax(outputs.detach().numpy()[idx])]  # part1: neutral/positive/negative

      ################################part 2########################################
      results_esg = nlp_esg(sents)
      esg_class_detail = results_esg[0]['label']  # part2: 九大分類

      if esg_class_detail == 'Climate Change' or  esg_class_detail == 'Natural Capital' or esg_class_detail == 'Pollution & Waste':
        esg_class = 'E'
      elif esg_class_detail == 'Human Capital' or  esg_class_detail == 'Product Liability' or esg_class_detail == 'Community Relations':
        esg_class = 'S'
      elif esg_class_detail == 'Corporate Governance' or  esg_class_detail == 'Business Ethics & Values':
        esg_class = 'G'
      else:
        esg_class = 'non_ESG'

      #############################整合part1 and part2##################################
      # 共有12種情況

      if sentiment == 'neutral':
        ESG_reports_df.loc[i, 'neutral'] += 1
      elif sentiment == 'positive':
        tweet_stock_cluster_df.loc[i, 'positive'] += 1
      else:
        tweet_stock_cluster_df.loc[i, 'negative'] += 1
      # print(s, '------', labels[np.argmax(outputs.detach().numpy()[idx])])

In [ ]:
# 例子2：利用finbert_esg9classification將ESG句子分類為9大面向
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert_esg = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories',num_labels=9)  # 選用finbert之9大ESG分類模型
tokenizer_esg = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')  # pre-train
nlp_esg = pipeline("text-classification", model=finbert_esg, tokenizer=tokenizer_esg)

results_esg = nlp_esg(['For 2002,', 'our total net emissions were approximately 60 million metric tons of CO2 equivalents for all businesses and operations we have ﬁnancial interests in, based on its equity share in those businesses and operations.'])
print(results) # [{'label': 'Climate Change', 'score': 0.9955655932426453}]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'Non-ESG', 'score': 0.985768735408783}, {'label': 'Climate Change', 'score': 0.9956915974617004}]


In [ ]:
results[0]['label']

'Non-ESG'

In [ ]:
# 將文字一句一句拆分 (應該改用標點符號切分，待改！！！)
text_r = text.replace('\n', ' ')  # 去掉換行符號，並用空格取代
print(text_r)

text_splits = text_r.split('.')
print(text_splits)


∕ 2 0 2 2 V I T A F A C T A N O N V E R B A LIFE T C C S U S T A I N A B I L I T Y R E P O R T T E R R A ｜ S O C I E T A S ｜ V I T ACOVER STORY V I T A F A C T A N O N The totem of water flow and greenery intertwined — ANKH — is an Egyptian hieroglyphic symbol that represents “life.” ANKH fused amidst all things on earth and flowing water, exhibiting the energy and pulse of life, as all life lives on in the natural circularity. V E R B A The TCC Report this year is themed with “In Service of Life.” Extension and protection of life constitutes the ultimate goal of all existence.TABLE OF CONTENTS Chairman’s Address 03 3.2＿Alternative Fuels & Materials 47 CHAPTER 7 EMPLOYEE BENEFITS TCC Commitments 3.3＿Industrial Symbiosis Ecosphere 50 7.1＿Special Column—Workplace of Diversity and Inclusion 96 Global Operations 05 OYAK & Cimpor: Overseas Cement Businesses 51 7.2＿Sustainable Talent Cultivation Strategies 97 3 Core Businesses & 10 Industrial Services 06 7.3＿Remuneration & Benefits 100 Total

In [ ]:
for i, txt in enumerate(text_splits):
  if len(text_splits[i]) > 500:
    print(i, len(text_splits[i]), text_splits[i])

103 1219 A CAMEROON CÔTE D'IVOIRE NETHERLANDS FRANCE ITALY SPAIN PORTUGAL TURKEY AUSTRALIA AND ENERGY E-ONE MOLI NHΩA CIMPOR (IVORY COAST) NHΩA NHΩA NHΩA CIMPOR OYAK ÇIMENTO NHΩA REVENUE RATIO ENERGY CAMEROUN CIMPOR CÔTE PORTUGAL (CANADA) KRIBI D'IVOIRE ABIDJAN HOLDINGS 29% Amsterdam Operation Headquarters RESOURCE NETHERLANDS RECYCLING EUROPE TAIWAN CEMENT(DUTCH) HOLDINGS REVENUE RATIO ASIA 68% NORTH AMERICA AFRICA Operation Headquarters ① OTHERS ④ ③ ① ⑤ ② REVENUE RATIO ⑦⑧⑥ ⑩ ⑨ ⑱ AUSTRALIA ⑪ ⑫⑭ ⑦⑲ 3% ⑬ ㉔ ② ⑤ MAINLAND ① ⑭ Hangzhou Operation Headquarters ② ⑮ ⑯ CHINA ⑰ ⑯ TAIWAN ③ ⑦ ⑮ ⑬ ⑪ ⑫ ⑧ ① ⑱③⑰ ⑲ CEMENT PLANT ① ⑫ L KI AA IO LIN ⑬ING A N② SH J UU NR O ⑭NG G ③ UA NYI GN AG ND ⑮E ④ XU L YO ON NG GS H ⑯AN H U⑤ AY S INH GA O ⑰GU CA HN O ⑥ NG G QU INIG GA ⑱NG S ⑦ UA OH U ⑲AIH HU OA P I⑧ NG JINGZHOU ⑨ BAOSHAN ⑩ ANNING ⑪ GANGAN ⑳ ⑨ ⑩ ⑤ ④ ㉒㉑ ⑥ RMC PLANT ① TAIPEI ② TUCHENG ③ GUISHAN ④ TAOYUAN ⑤ TAOYUAN 2 ⑥ BADE ⑦ ZHONGLI ⑧ ZHONGLI 2 ⑨ HSINCHU ⑩ YILAN ⑪ TAICHUNG ⑫ SHENGANG ② ⑥ ③ ④㉓ ⑬ DADU ⑭ TAIP

In [ ]:
# 砍掉>500的句子
text_splits_adj = []
for i, txt in enumerate(text_splits):
  if len(text_splits[i]) <= 500:
    text_splits_adj.append(text_splits[i])

print(len(text_splits_adj))

2659


In [ ]:
# double check
for i, txt in enumerate(text_splits_adj):
  if len(text_splits_adj[i]) > 500:
    print(i, len(text_splits_adj[i]), text_splits_adj[i])

In [ ]:
# 測試--先用200字切分
def split_text_into_chunks(txt, chunk_size):
    chunks = [txt[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

# 要切分的文字


# 切分成每 200 個字一段
chunk_size = 200
text_chunks = split_text_into_chunks(text_r, chunk_size)
print(text_chunks)
# 打印切分後的結果
#for i, chunk in enumerate(text_chunks):
    #print(f"段 {i + 1}: {chunk}")

# check
for i, txt in enumerate(text_chunks):
  if len(text_chunks[i]) > 500:
    print(i, len(text_chunks[i]), text_chunks[i])
print('OK')

['∕ 2 0 2 2 V I T A F A C T A T E R R N O N A ｜ S O C V E R B A I E T A S LIFE ｜ T C C S U S T A I N A B I L I T Y R E P O R T V I T ACOVER STORY V I T A F A C T A N O N The totem of water flow and gree', 'nery intertwined — ANKH — is an Egyptian hieroglyphic symbol that represents “life.” ANKH fused amidst all things on earth and flowing water, exhibiting the energy and pulse of life, as all life lives', ' on in the natural circularity. V E R B A The TCC Report this year is themed with “In Service of Life.” Extension and protection of life constitutes the ultimate goal of all existence.TABLE OF CONTENT', 'S Chairman’s Address 03 3.2＿Alternative Fuels & Materials 47 CHAPTER 7 EMPLOYEE BENEFITS TCC Commitments 3.3＿Industrial Symbiosis Ecosphere 50 7.1＿Special Column—Workplace of Diversity and Inclusion 9', '6 Global Operations 05 OYAK & Cimpor: Overseas Cement Businesses 51 7.2＿Sustainable Talent Cultivation Strategies 97 3 Core Businesses & 10 Industrial Services 06 7.3＿Remuneration & 

In [ ]:
# 安裝transformers
%pip install transformers

# import套件
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

# Download and load the fine-tuned FinBERT sentiment classification model and the vocabulary (FinVocab, the finance vocabulary discussed earlier) from Hugging Face.
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)  # fine-tuned FinBERT sentiment classification model
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')  # the vocabulary (FinVocab, the finance vocabulary)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.9 MB/s eta 0:00:00


In [ ]:
# text_splits_adj:切分好的句子
i = 0  # start index
last_round = False  # 若最後一round做完，跳出迴圈

while True:
  # check是否做完了
  if last_round == True:
    break  # 跳出迴圈

  # check是否已到最後一round
  if i + 50 > len(text_splits_adj):  # 這一round未滿50句
    round_last_index = len(text_splits_adj) - 1  # 最後一句的index
    last_round = True  # 更改為做完最後一round
  else:
    round_last_index = i+50  # 往上加50句


  inputs = tokenizer(text_splits_adj[i:round_last_index], return_tensors="pt", padding=True)  # tokenize the sentences (pt:PyTorch輸出轉換形式)
  outputs = finbert(**inputs)[0]  # 輸出 process them in FinBERT


  # Assign each sentence to its most probable sentiment category.
  labels = {0:'neutral', 1:'positive', 2:'negative'}

  #for idx, sent in enumerate(text_splits_adj[i:round_last_index]):  # idx=index sent=sentence
    #print(idx, sent, '------', labels[np.argmax(outputs.detach().numpy()[idx])])

  i += 50  # 下一round (每批做50句)
print(last_round)